REGULARIZACIÓN PARA REGRESIÓN LINEAL| 
-----------------------

Se han recopilado datos sociodemográficos y de recursos de salud por condado en los Estados Unidos y queremos descubrir si existe alguna relación entre los recursos sanitarios y los datos sociodemográficos.

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LassoCV, RidgeCV

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")
pd.set_option('display.max_rows',110)
pd.set_option('display.max_columns',110)
data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,40-49,40-49 y/o % of total pop,50-59,50-59 y/o % of total pop,60-69,60-69 y/o % of total pop,70-79,70-79 y/o % of total pop,80+,80+ y/o % of total pop,White-alone pop,% White-alone,Black-alone pop,% Black-alone,Native American/American Indian-alone pop,% NA/AI-alone,Asian-alone pop,% Asian-alone,Hawaiian/Pacific Islander-alone pop,% Hawaiian/PI-alone,Two or more races pop,% Two or more races,POP_ESTIMATE_2018,N_POP_CHG_2018,GQ_ESTIMATES_2018,R_birth_2018,R_death_2018,R_NATURAL_INC_2018,R_INTERNATIONAL_MIG_2018,R_DOMESTIC_MIG_2018,R_NET_MIG_2018,Less than a high school diploma 2014-18,High school diploma only 2014-18,Some college or associate's degree 2014-18,Bachelor's degree or higher 2014-18,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a high school diploma only 2014-18,Percent of adults completing some college or associate's degree 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,POVALL_2018,PCTPOVALL_2018,PCTPOV017_2018,PCTPOV517_2018,MEDHHINC_2018,CI90LBINC_2018,CI90UBINC_2018,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Active Physicians per 100000 Population 2018 (AAMC),Total Active Patient Care Physicians per 100000 Population 2018 (AAMC),Active Primary Care Physicians per 100000 Population 2018 (AAMC),Active Patient Care Primary Care Physicians per 100000 Population 2018 (AAMC),Active General Surgeons per 100000 Population 2018 (AAMC),Active Patient Care General Surgeons per 100000 Population 2018 (AAMC),Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),ICU Beds_x,Total Population,Population Aged 60+,Percent of Population Aged 60+,COUNTY_NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,county_pop2018_18 and older,anycondition_prevalence,anycondition_Lower 95% CI,anycondition_Upper 95% CI,anycondition_number,Obesity_prevalence,Obesity_Lower 95% CI,Obesity_Upper 95% CI,Obesity_number,Heart disease_prevalence,Heart disease_Lower 95% CI,Heart disease_Upper 95% CI,Heart disease_number,COPD_prevalence,COPD_Lower 95% CI,COPD_Upper 95% CI,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,7582,13.636445,7738,13.917016,5826,10.478229,4050,7.284042,2014,3.622237,42660,76.725239,10915,19.630942,267,0.480207,681,1.224798,62,0.111509,1016,1.827305,55601,158,455,11.8,9.6,2.2,0.0,0.7,0.6,4204,12119,10552,10291,11.3,32.6,28.4,27.7,7587,13.8,19.3,19.5,59338,53628,65048,25957,25015,942,3.6,59338,119.0,217.1,196.7,77.2,71.2,7.6,6.9,28.859137,6.085786,1.148905,25.992561,21.249061,72.142154,6,55036,10523,19.1,Autauga,Alabama,1,1,42438,47.6,45.4,49.4,20181,35.8,34.2,37.3,15193,7.9,7.2,8.7,3345,8.6,7.3,9.9,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,27338,12.539102,29986,13.753658,29932,13.728890,20936,9.602701,9368,4.296814,190301,87.285228,19492,8.940382,1684,0.772399,2508,1.150343,146,0.066966,3891,1.784682,218022,5403,2190,10.5,10.3,0.1,0.5,24.3,24.8,14310,40579,46025,46075,9.7,27.6,31.3,31.3,21069,9.8,13.9,13.1,57588,54437,60739,93849,90456,3393,3.6,57588,115.5,217.1,196.7,77.2,71.2,7.6,6.9,113.162114,23.863512,4.505074,101.921730,83.321572,282.882982,51,203360,53519,26.3,Baldwin,Alabama,1,3,170912,40.2,38.2,42.3,68790,29.7,28.4,31.0,50761,7.8,7.0,8.7,13414,8.6,7.2,10.1,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,3074,12.354809,3278,13.174712,3076,12.3

Análisis descriptivo de las variables: 

In [3]:
data.shape

(3140, 108)

In [4]:
pd.set_option('display.max_rows',110)
pd.set_option('display.max_columns',110)  # Max number of columns to display
data.describe().T

,count,mean,std,min,25%,50%,75%,max
fips,3140.0,30401.640764,15150.559265,1001.000000,18180.500000,29178.000000,45081.500000,5.604500e+04
TOT_POP,3140.0,104189.412420,333583.395432,88.000000,10963.250000,25800.500000,67913.000000,1.010552e+07
0-9,3140.0,12740.302866,41807.301846,0.000000,1280.500000,3057.000000,8097.000000,1.208253e+06
0-9 y/o % of total pop,3140.0,11.871051,2.124081,0.000000,10.594639,11.802727,12.951840,2.546068e+01
19-Oct,3140.0,13367.976752,42284.392134,0.000000,1374.500000,3274.000000,8822.250000,1.239139e+06
10-19 y/o % of total pop,3140.0,12.694609,1.815044,0.000000,11.674504,12.687422,13.659282,2.330437e+01
20-29,3140.0,14469.331529,49577.725396,0.000000,1263.750000,3108.000000,8976.250000,1.557073e+06
20-29 y/o % of total pop,3140.0,12.283979,3.126297,0.000000,10.496774,11.772649,13.182260,3.757020e+01
30-39,3140.0,13916.490127,48990.951710,11.000000,1232.750000,3000.500000,8314.250000,1.501844e+06
30-39 y/o % of total pop,3140.0,11.751535,1.696599,6.092789,10.689322,11.580861,12.639379,2.222513e+01


In [5]:
data.isna().sum().sum()

0

Factorización/numeración de variables categóricas:

In [6]:
cat_data = data.select_dtypes(include=['object']).columns
cat_data

Index(['COUNTY_NAME', 'STATE_NAME'], dtype='object')

In [7]:
data['COUNTY_NAME_N'] = pd.factorize(data.COUNTY_NAME)[0]
data['STATE_NAME_N'] = pd.factorize(data.STATE_NAME)[0]

In [8]:
data.drop(cat_data, axis=1, inplace=True)

****Selección de Características****

Tendremos que seleccionar las mejores características para pasarle al modelo.


Busqueda de correlaciones: 

Variable Objetivo = Heart disease_prevalence

In [9]:
# Correlaciones

corr_data = data.corr()
np_corr = np.where(corr_data >= 0.15, corr_data, 0)
corr = pd.DataFrame(np_corr, columns=corr_data.columns, index=corr_data.index)

hd_np_corr = corr['Heart disease_prevalence']
corr['Heart disease_prevalence'][corr['Heart disease_prevalence'] > 0]


print(f"Hay: {len(hd_np_corr[(hd_np_corr > 0)])} variables predictoras que presentan una alta correlación con la variable objetivo")





Hay: 31 variables predictoras que presentan una alta correlación con la variable objetivo


In [10]:
hd_np_corr[(hd_np_corr > 0) & (hd_np_corr < 0.9)]

50-59 y/o % of total pop                                          0.282467
60-69 y/o % of total pop                                          0.516140
70-79 y/o % of total pop                                          0.641682
80+ y/o % of total pop                                            0.472387
R_death_2018                                                      0.683237
Percent of adults with less than a high school diploma 2014-18    0.474498
Percent of adults with a high school diploma only 2014-18         0.602560
PCTPOVALL_2018                                                    0.546347
PCTPOV017_2018                                                    0.665902
PCTPOV517_2018                                                    0.650718
Unemployment_rate_2018                                            0.388944
Percent of Population Aged 60+                                    0.595762
anycondition_prevalence                                           0.784088
anycondition_Lower 95% CI

In [11]:
#Formamos los conjuntos de Train y Test

x = data.drop('Heart disease_prevalence', axis=1)
y= data['Heart disease_prevalence']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [12]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [13]:
# Crear un modelo XGBoost Regressor

model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train_sc, y_train)
feature_importances = model.feature_importances_

feature_names = x.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True))

# Imprime la importancia de las características (número adimensional, las características se comparan entre sí en términos de su contribución relativa a la predicción)
for feature, importance in sorted_feature_importance.items():
    print(f'Feature: {feature}, Importance: {importance*100}')

Feature: Heart disease_Upper 95% CI, Importance: 76.37372016906738
Feature: Heart disease_Lower 95% CI, Importance: 23.263318836688995
Feature: CKD_prevalence, Importance: 0.01720261061564088
Feature: COPD_Upper 95% CI, Importance: 0.011125325545435771
Feature: CKD_Upper 95% CI, Importance: 0.009181616042042151
Feature: CKD_Lower 95% CI, Importance: 0.008814619650365785
Feature: anycondition_Upper 95% CI, Importance: 0.007028701656963676
Feature: High school diploma only 2014-18, Importance: 0.006176515307743102
Feature: diabetes_Upper 95% CI, Importance: 0.005868117659701966
Feature: 50-59, Importance: 0.005847561260452494
Feature: PCTPOV017_2018, Importance: 0.005832591705257073
Feature: COPD_prevalence, Importance: 0.00559034087928012
Feature: diabetes_number, Importance: 0.005377035995479673
Feature: STATE_FIPS, Importance: 0.005363930176827125
Feature: R_NATURAL_INC_2018, Importance: 0.005222165418672375
Feature: Percent of adults with a bachelor's degree or higher 2014-18, Import

In [14]:
def get_vars_to_delete(d_imp, corr, imp_cut = 0.1, corr_cut = 0.15):
  ls_vars_low_corr = corr[corr['Heart disease_prevalence'].abs()<=corr_cut].index
  return [var for var in ls_vars_low_corr if d_imp[var]<=imp_cut]


vars_to_drop = get_vars_to_delete(sorted_feature_importance, corr_data, imp_cut=0.1, corr_cut=0.15)
vars_to_drop

['fips',
 '% White-alone',
 '% Black-alone',
 '% NA/AI-alone',
 '% Hawaiian/PI-alone',
 '% Two or more races',
 'R_DOMESTIC_MIG_2018',
 'R_NET_MIG_2018',
 'Active General Surgeons per 100000 Population 2018 (AAMC)',
 'Active Patient Care General Surgeons per 100000 Population 2018 (AAMC)',
 'STATE_FIPS',
 'CNTY_FIPS',
 'COUNTY_NAME_N',
 'STATE_NAME_N']

In [15]:
#Eliminacion de variables:
X_train_sc.drop(vars_to_drop, axis = 1, inplace=True)
X_test_sc.drop(vars_to_drop, axis = 1, inplace=True)

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [16]:
#Variables que apuntan a lo mismo que otras variables dentro de DF, pero con distintas escalas. Pueden probocar multicolinearidad. 

var_dup = ["20-29", "30-39", "40-49", "50-59", "50-59", "70-79", "80+" , "White-alone pop", "Asian-alone pop" ]

In [17]:
#Para corregir "numpy.ndarray' object has no attribute 'drop" y elimiar las variables del modelo: 

# Convertir arrays de NumPy a DataFrames
X_train_df = pd.DataFrame(X_train_sc, columns=x.columns)
X_test_df = pd.DataFrame(X_test_sc, columns=x.columns)

# Eliminar las variables con menos relevantes
X_train_df = X_train_df.drop(vars_to_drop, axis=1)
X_test_df = X_test_df.drop(vars_to_drop, axis=1)

# Eliminar las variables con colinearidad
X_train_df = X_train_df.drop(var_dup, axis=1)
X_test_df = X_test_df.drop(var_dup, axis=1)

# Convertir DataFrames de vuelta a arrays de NumPy
X_train_sc = X_train_df.values
X_test_sc = X_test_df.values



***MODELIZADO CON LINEAR REGRESSION***

Ya hecha la selección de características ahora voy a instanciar y modelizar los datos. 

In [18]:
model = LinearRegression()

resultado = model.fit(X_train_sc, y_train)

In [19]:
#Intercepto del Modelo

model.intercept_

8.598938428708829

In [20]:
# El coeficiente de las Variables Predictoras

coef = dict(zip(X_train_df.columns, model.coef_)) 
dict(sorted(coef.items(), key=lambda x: x[1], reverse=True))

{'TOT_POP': 45183119338.9959,
 'Heart disease_Upper 95% CI': 0.9454989433288574,
 'Heart disease_Lower 95% CI': 0.7888240814208984,
 'diabetes_prevalence': 0.49082422256469727,
 'COPD_prevalence': 0.2189655303955078,
 'anycondition_prevalence': 0.1757490634918213,
 'COPD_number': 0.04532146453857422,
 'R_NATURAL_INC_2018': 0.0353693962097168,
 'Obesity_Upper 95% CI': 0.029460906982421875,
 'Population Aged 60+': 0.028382301330566406,
 'CKD_Lower 95% CI': 0.023221969604492188,
 'R_death_2018': 0.02225971221923828,
 'PCTPOV517_2018': 0.017370223999023438,
 'MEDHHINC_2018': 0.013826847076416016,
 'Obesity_number': 0.01359415054321289,
 'Total Active Patient Care Physicians per 100000 Population 2018 (AAMC)': 0.011997222900390625,
 'POVALL_2018': 0.009569168090820312,
 'CKD_number': 0.009442806243896484,
 'Active Primary Care Physicians per 100000 Population 2018 (AAMC)': 0.008472442626953125,
 'CKD_prevalence': 0.0067119598388671875,
 'Median_Household_Income_2018': 0.006696224212646484,


In [21]:
yhat_train = model.predict(X_train_sc)
yhat_test = model.predict(X_test_sc)

In [22]:
def get_metrics(yhat, y_test, yhat_train, y_train):
  metrics_train = r2_score(y_train, yhat_train), median_absolute_error(y_train, yhat_train), mean_absolute_percentage_error(y_train, yhat_train)*100
  metrics_test = r2_score(y_test, yhat), median_absolute_error(y_test, yhat), mean_absolute_percentage_error(y_test, yhat)*100
  metrics_diff = list(map(lambda x: x[1]-x[0], zip(metrics_train, metrics_test)))
  return pd.DataFrame(data=[metrics_train, metrics_test, metrics_diff], columns=['R2', 'Median AE', 'MAPE'], index=['Train set', 'Test set', 'Diferencia'])

In [23]:
get_metrics(yhat_test, y_test, yhat_train, y_train)

,R2,Median AE,MAPE
Train set,0.999489,0.026587,0.389062
Test set,0.999510,0.027356,0.398426
Diferencia,0.000020,0.000768,0.009364


***REGULARIZACIÓN***

In [26]:
# Regularización con Lasso
# Búsqueda del mejor valor de alpha

m_lasso = LassoCV(alphas=np.logspace(-6, 6, 10), cv=5, random_state=42, n_jobs=-1).fit(X_train_sc, y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0429962723810184, tolerance: 0.575406273354565
  model = cd_fast.enet_coordinate_descent_gram(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4617822779763694, tolerance: 0.575406273354565
  model = cd_fast.enet_coordinate_descent_gram(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0664458934448977, tolerance: 0.575557991507431
  model = cd_fast.enet_coordinate_descent_gram(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_co

In [27]:
best_alpha = m_lasso.alpha_
print("Mejor valor de alpha:", best_alpha)

Mejor valor de alpha: 0.00046415888336127773


In [28]:
y_pred_train_lasso = m_lasso.predict(X_train_sc)
y_pred_test_lasso = m_lasso.predict(X_test_sc)

In [29]:
get_metrics(y_pred_test_lasso, y_test, y_pred_train_lasso, y_train)

,R2,Median AE,MAPE
Train set,0.999439,0.026635,0.416541
Test set,0.999488,0.026541,0.413687
Diferencia,0.000050,-0.000094,-0.002854


In [31]:
len([x for x in m_lasso.coef_ if np.abs(x)==0])/len(m_lasso.coef_)*100
print(f'El porcentaje de variables canceladas es: {round(len(m_lasso.coef_[np.abs(m_lasso.coef_)==0])/len(m_lasso.coef_)*100,2)}%')

El porcentaje de variables canceladas es: 74.12%


In [32]:
print(f'El modelo utiliza {len(m_lasso.coef_[np.abs(m_lasso.coef_)>0])} variables.')

El modelo utiliza 22 variables.
